In [7]:
# 如果本地没有安装工具包，请去掉“#”，再执行
# !pip install -i https://mirrors.aliyun.com/pypi/simple/ xlwt

**<font color=blue>Excel颜色索引对照表</font>**<br/>
<img src="images/Excel颜色对照表.png" align="left"></img>

In [11]:
# 导入Excel读写工具包
import xlrd
import xlwt

# 从（from）本地 xlwt_styles.py（.py扩展名省略）文件中，导入（import）创建样式的函数（create_style）
from xlwt_styles import create_style

# 定义两个变量（xls_filename_rd、xls_filename_wt）
xls_filename_rd = "./files/Excel_工资单明细.xls" # 读取的 Excel 文件名
xls_filename_wt = "./files/Excel_员工工资单_02.xls" # 新建的 Excel 文件名
# *******************************
# 命名最好要有意义，但不是强制，只有符合变量命名规则即可。
# xls:Excel扩展名  filename:文件名  rd:read读取  wt:write写入
# *******************************

# 读取 Excel（xls_filename_rd 变量刚才已经定义），得到工作簿，赋值给变量 workbook_rd
workbook_rd = xlrd.open_workbook(xls_filename_rd)

# 获取第一个工作表，赋值给变量 sheet_rd
sheet_rd = workbook_rd.sheet_by_index(0)

# 获取工作表中表格的所有字段（第一行是字段名），记住后面会用到
field_names = sheet_rd.row_values(0)
print(field_names)  # 打印一下看看字段名

['月份', '部门', '姓名', '出勤工资', '绩效奖金', '税前补发', '生活补助', '应发', '养老', '医疗', '失业', '计税工资', '个人所得税', '扣款合计', '实发']


In [12]:
# 创建两个“空”列表变量
staff_names = [] # 员工姓名列表
staff_values_list = [] # 员工数据明细列表（这个就是列表中的列表）

# 因为第一行（索引=0）是字段名，所以从第二行（索引=1）开始读数据
for row_index in range(1, sheet_rd.nrows):
    values = sheet_rd.row_values(row_index)  # 获取员工的数据列表
    # print(values)           # 可以删除注释，打印这一行数据列表看看
    staff_name = values[2]    # 第三列是姓名
    # print(staff_name)       # 可以删除注释，打印一下看看
    staff_names.append(staff_name)  # append
    staff_values_list.append(values) # 列表中可以再包含列表 [[...],[...]]

# 打印读取的员工姓名列表、员工数据明细列表
# print("员工姓名列表：",staff_names)           # 可以删除注释，打印一下看看
# print("员工数据明细列表：",staff_values_list) # 可以删除注释，打印一下看看

In [13]:
# 创建写入工作簿对象
workbook_wt = xlwt.Workbook(encoding='utf-8')

# 看看要生成的表格样式，共有4个
# 合并单元格的样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=居中）
merge_style = create_style(back_color=57, font_color=1, font_size=12, horz=2)  # 标题显示用黄色背景图案
# 字段样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=靠右）
field_style = create_style(back_color=57, font_color=1, font_size=12, horz=3)
# 数据值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=居中）
value_style = create_style(back_color=1, font_color=0, font_size=12, horz=2)
# 金额值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=靠右，人民币格式）
rmb_style = create_style(back_color=1, font_color=0, font_size=12, horz=3, num_format_str="￥#,##0.00;￥-#,##0.00")

# 循环遍历每个员工的姓名
for staff_index, staff_name in enumerate(staff_names):
    # 创建工作表，工作表名就是员工姓名
    sheet_wt = workbook_wt.add_sheet(staff_name)

    # 获取员工的数据明细
    staff_data = staff_values_list[staff_index]
    # 遍历字段和数据
    for field_index, field_name in enumerate(field_names):
        if field_index < 14:
            # 第3列为字段名，第4列为数据
            sheet_wt.write(r=field_index + 1, c=2, label=field_name, style=field_style)

        value = staff_data[field_index]
        # 从 0 开始，大于第2个字段，使用RMB货币样式

        if field_index > 2:
            sheet_wt.write(r=field_index + 1, c=3, label=value, style=rmb_style)
        else:
            sheet_wt.write(r=field_index + 1, c=3, label=value, style=value_style)

    # 设置列宽
    sheet_wt.col(1).width = 256 * 12
    sheet_wt.col(2).width = 256 * 15
    sheet_wt.col(3).width = 256 * 20

    sheet_wt.write_merge(r1=1, r2=8, c1=1, c2=1, label="收\n入", style=merge_style)
    sheet_wt.write_merge(r1=9, r2=14, c1=1, c2=1, label="扣\n款", style=merge_style)
    sheet_wt.write_merge(r1=15, r2=15, c1=1, c2=2, label="实发", style=field_style)
# import time
# time.sleep(2)
# 保存
workbook_wt.save(xls_filename_wt)

print("文件 %s 生成成功" % xls_filename_wt)

文件 ./files/Excel_员工工资单_02.xls 生成成功
